# Nuclei Segmentation

Let's begin with nuclei segmentation.
We can either segment the DAPI channel and then filter out the healthy nuclei, segment only the healthy nuclei or go for the ISL1 channel that has the label for the nuclei of interest.

Let's select an image and then crop out a small piece of it to make testing easier.

In [1]:
from pathlib import Path

import numpy as np
from czifile import CziFile
from napari import Viewer
from scipy.ndimage import distance_transform_edt

In [2]:
DATA_DIR = Path("../data/NS#1_Healhty-DAPI, WGA, ISL1, G3BP.czi")
image_handle = CziFile(DATA_DIR)
img = np.squeeze(image_handle.asarray())[3, :, 1000:1500, 1000:1500]

In [3]:
scale = {
    values_dict["Id"]: values_dict["Value"] * 10**6
    for values_dict in image_handle.metadata(raw=False)["ImageDocument"]["Metadata"][
        "Scaling"
    ]["Items"]["Distance"]
}
spacing = (scale["Z"], scale["Y"], scale["X"])

## Histogram Equalization

We should first try some histogram equalization as the intensity values ar every different across planes.
I am not sure this will solve some of the saturation problems though.

In [4]:
from skimage.exposure import equalize_adapthist  # noqa

In [7]:
kernel = (50 / scale["Z"], 50 / scale["Y"], 50 / scale["X"])

equalized_img = equalize_adapthist(
    img,
    kernel_size=kernel,
    clip_limit=0.05,
)

In [8]:
viewer = Viewer()
viewer.add_image(img, scale=spacing, visible=False)
viewer.add_image(equalized_img, scale=spacing, visible=True)

<Image layer 'equalized_img' at 0x7d90a657f680>

## Cellpose

Let's begin by trying out cellpose.

In [9]:
from cellpose import models  # noqa
from cellpose.io import logger_setup  # noqa

logger_setup()

2024-07-01 11:56:06,682 [INFO] WRITING LOG OUTPUT TO /home/biif/.cellpose/run.log
2024-07-01 11:56:06,682 [INFO] 
cellpose version: 	3.0.10 
platform:       	linux 
python version: 	3.12.4 
torch version:  	2.3.1+cu121


(<Logger cellpose.io (INFO)>, PosixPath('/home/biif/.cellpose/run.log'))

In [10]:
model = models.Cellpose(model_type="nuclei")

2024-07-01 11:56:06,696 [INFO] >>>> using CPU
2024-07-01 11:56:06,699 [INFO] >> nuclei << model set to be used
2024-07-01 11:56:06,842 [INFO] >>>> loading model /home/biif/.cellpose/models/nucleitorch_0
2024-07-01 11:56:06,984 [INFO] >>>> model diam_mean =  17.000 (ROIs rescaled to this size during training)


After a couple attempts with several algorithms, I thought there might be issues with all the saturated nuclei.
To circunvent this, I made a mask for those pixels, calculated the distance transform and added this to the original image in order to have some values here.

In [ ]:
saturated_mask = img[3] == 255
distance_saturated = np.asarray(
    [distance_transform_edt(this_plane) for this_plane in saturated_mask]
)

In [ ]:
img = img.astype(float)
img[3] = img[3] + 10 * distance_saturated
img[3] = img[3] / (np.max(img[3]) + 10)

In [11]:
anisotropy = scale["Z"] / scale["X"]
print(anisotropy)

5.505521313330107


In [21]:
diameter = 10 / scale["X"]
print(diameter)

72.86719385289848


In [22]:
masks, flows, styles, diams = model.eval(
    [equalized_img],
    batch_size=8,
    diameter=diameter,
    z_axis=0,
    do_3D=True,
    channels=[0, 0],
    # stitch_threshold=0.5,
    normalize=False,
    anisotropy=anisotropy,
)

2024-07-01 13:27:34,043 [INFO] channels set to [0, 0]
2024-07-01 13:27:34,045 [INFO] ~~~ FINDING MASKS ~~~
2024-07-01 13:27:34,046 [INFO] multi-stack tiff read in as having 110 planes 1 channels
2024-07-01 13:27:34,348 [INFO] running YX: 110 planes of size (500, 500)
2024-07-01 13:27:52,079 [INFO] 100%|##########| 4/4 [00:17<00:00,  4.43s/it]
2024-07-01 13:27:53,003 [INFO] running ZY: 500 planes of size (110, 500)
2024-07-01 13:28:57,117 [INFO] 100%|##########| 16/16 [01:04<00:00,  4.00s/it]
2024-07-01 13:28:59,172 [INFO] running ZX: 500 planes of size (110, 500)
2024-07-01 13:30:01,712 [INFO] 100%|##########| 16/16 [01:02<00:00,  3.91s/it]
2024-07-01 13:30:03,293 [INFO] network run in 149.03s
2024-07-01 13:30:37,160 [INFO] masks created in 33.87s
2024-07-01 13:30:39,838 [INFO] >>>> TOTAL TIME 185.80 sec


In [23]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(masks[0], scale=spacing)

<Labels layer 'Labels' at 0x7d902175f230>

It's working quite well for some nuclei.
Some are still over segmented and some others are not segmented.

## StarDist

Maybe stardist might work.
Not tested yet.

## Clesperanto Voronoi Otsu

Maybe Clesperanto's Voronoi and Otsu labelling works well here.

In [ ]:
import pyclesperanto_prototype as cle  # noqa

cle.available_device_names()

In [ ]:
input_gpu = cle.push(img[3])
input_gpu

I was unable to make clesperanto work on the laptop.

## Thresholding

Let's go for typical thresholding techniques as nuclei might be separated enough to not need watershed or maybe a simple watershed.

In [18]:
from skimage.feature import peak_local_max  # noqa
from skimage.filters import median, scharr, threshold_otsu  # noqa
from skimage.morphology import ball, binary_erosion, dilation, disk  # noqa
from skimage.segmentation import watershed  # noqa

Let's begin with some median filtering to enhance a bit the edges.
I'm using my own footprint.

And then use Otsu thresolding.

In [55]:
pixel_resolution = 0.5 / scale["X"]

footprint = np.stack(
    [
        np.zeros_like(disk(pixel_resolution)),
        disk(pixel_resolution),
        np.zeros_like(disk(pixel_resolution)),
    ]
)
midpoint = footprint.shape[1] // 2
footprint[0, midpoint, midpoint] = 1
footprint[2, midpoint, midpoint] = 1

filtered_image = median(img, footprint=footprint)
threshold = threshold_otsu(filtered_image)
nuclei = filtered_image > threshold

In [8]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(nuclei, scale=spacing)

<Labels layer 'nuclei' at 0x7c8d8b380ce0>

Let's apply watershed to get a better separation between nuclei.
We will need the seeds.

In [64]:
footprint = np.stack([np.zeros_like(disk(1)), disk(1), np.zeros_like(disk(1))])

inter = binary_erosion(nuclei, footprint=footprint)

transformed = distance_transform_edt(inter, sampling=spacing)

distance_between_nuclei = 1.7  # in um
pixel_min_distance = np.floor(distance_between_nuclei / scale["X"]).astype(int)

maxima = peak_local_max(transformed, min_distance=pixel_min_distance)

In [65]:
print(pixel_min_distance, len(maxima))

13 77


We need an image for the edges of the nuclei to limit the watershed.

In [58]:
edges = scharr(filtered_image)

In [59]:
viewer = Viewer()
viewer.add_image(transformed, scale=spacing)
viewer.add_image(edges, scale=spacing)
viewer.add_points(
    maxima,
    name="points",
    scale=spacing,
    size=40,
    n_dimensional=True,  # points have 3D "extent"
    face_color="red",
)

<Points layer 'points' at 0x7c8cf3b9cfb0>

In [68]:
markers = np.zeros(img.shape, dtype=np.uint32)
marker_indices = tuple(np.round(maxima).astype(int).T)
markers[marker_indices] = np.arange(len(maxima)) + 1
markers = dilation(markers, ball(5))

segmented = watershed(
    -transformed,
    markers,
    mask=nuclei,
)

In [69]:
viewer = Viewer()
viewer.add_image(img, scale=spacing)
viewer.add_labels(segmented, scale=spacing)

<Labels layer 'segmented' at 0x7c8cb9a0bef0>

It's not the best and should be improved, but it's not that bad.